# Prepare Notebook Environment for Strands Agents

In this section we prepare this notebook environment with the necessary dependencies to create any agents using the Strands framework.

# Prerequisites

1. If you are running the notebook in your own account and not on as part of an AWS hosted event, follow the **Deployment** instructions from the [Self Paced](https://catalog.us-east-1.prod.workshops.aws/workshops/0bf501fc-357d-43b3-b758-b44f9b0e7d49/en-US/introduction/access-and-navigate-aws-account/self-paced) workshop to setup the following dependencies in your account:
    - Networking infrastructure (VPC, Subnets, etc.)
    - Amazon Redshift Database
    - Amazon Bedrock Knowledge Base
2. Configure Bedrock Model Access to the following models:
    - Amazon
        - Titan Embeddings G1 - Text
    - Anthropic
        - Anthropic Claude 3.7 Sonnet
3. Manually sync the Amazon Bedrock Knowledge Base:
    - Navigate to the Amazon Bedrock dashboard via AWS Console search
    - Select the option icon (top left) to open the navigation bar
    - Select "Knowledge bases" under the "Builder tools" tab
    - Choose the Knowledgebase created by the AWS CloudFormation template
    - Scroll to the "Data Source" option box
    - Select the data source (radio button) and Select Sync
4. Make sure your notebook execution role has the following managed policies:
    - AmazonBedrockFullAccess
    - AmazonRedshiftQueryEditor
    - AmazonS3FullAccess
    - AmazonSageMakerFullAccess
    - AWSLambda_FullAccess
    - AWSStepFunctionsFullAccess
    - IAMFullAccess
    - AWSCodeBuildAdminAccess
5. For protein design notebooks, additional AWS HealthOmics permissions will be automatically added by this setup notebook.

# Environment Setup

#### Run the pip commands below to install all needed packages

In [1]:
%pip install boto3 awscli botocore termcolor sagemaker --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.37.38 which is incompatible.
langchain-aws 0.2.19 requires boto3>=1.37.24, but you have boto3 1.37.1 which is incompatible.
sagemaker-studio-analytics-extension 0.2.0 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Ensure the boto3 version printed below is **1.37.1** or higher.

In [2]:
%pip show boto3

Name: boto3
Version: 1.37.1
Summary: The AWS SDK for Python
Home-page: https://github.com/boto/boto3
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: botocore, jmespath, s3transfer
Required-by: amazon-q-developer-jupyterlab-ext, amazon-sagemaker-sql-editor, amazon-sagemaker-sql-execution, amazon_sagemaker_jupyter_ai_q_developer, autogluon.common, autogluon.core, autogluon.multimodal, aws-glue-sessions, langchain-aws, PyAthena, redshift_connector, sagemaker, sagemaker-core, sagemaker-jupyterlab-extension-common, sagemaker-kernel-wrapper, sagemaker-mlflow, sagemaker-studio-analytics-extension, sagemaker-studio-dataengineering-extensions, sagemaker-studio-dataengineering-sessions, sagemaker_studio, sagemaker_studio_sparkmagic_lib, snowflake-connector-python
Note: you may need to restart the kernel to use updated packages.


#### Import Python libraries

In [3]:
# Standard Python libraries
import boto3
import sagemaker

# Import needed functions to create agent
from utils.bedrock_agent_helper import AgentsForAmazonBedrock
from utils.role_policy_helper import SageMakerRolePolicyChecker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


#### Extract account information needed for agent creation SageMaker role

In [4]:
# boto3 session
sts_client = boto3.client('sts')
session = boto3.session.Session()

# Account info
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
print(f"Account ID: {account_id}")
print(f"Region: {region}")

# Get the SageMaker session and execution role
sagemaker_session = sagemaker.Session()
role = sagemaker_session.get_caller_identity_arn()
print(f"SageMaker Execution Role: {role}")

Account ID: 229020249659
Region: us-east-1
SageMaker Execution Role: arn:aws:iam::229020249659:role/service-role/AmazonSageMaker-ExecutionRole-20250928T234110


#### Check if the SageMaker role has the required policies to run the notebooks

In [5]:
# Check and add required policies
role_checker = SageMakerRolePolicyChecker()
role_name = role.split('/')[-1]  # Extract role name from ARN

try:
    # Check existing policies
    role_checker.check_policies(role)
    print("All required managed policies are attached!")
except Exception as e:
    print(f"Missing policies: {e}")
    print("Please attach the missing policies manually or contact your administrator.")

# Add HealthOmics policy for protein design notebook
try:
    role_checker.attach_healthomics_policy(role_name)
    print("HealthOmics policy attached successfully!")
except Exception as e:
    print(f"Note: HealthOmics policy attachment: {e}")

Missing policies: Error retrieving attached policies: An error occurred (AccessDenied) when calling the ListAttachedRolePolicies operation: User: arn:aws:sts::229020249659:assumed-role/AmazonSageMaker-ExecutionRole-20250928T234110/SageMaker is not authorized to perform: iam:ListAttachedRolePolicies on resource: role AmazonSageMaker-ExecutionRole-20250928T234110 because no identity-based policy allows the iam:ListAttachedRolePolicies action
Please attach the missing policies manually or contact your administrator.
Note: HealthOmics policy attachment: Error creating policy: An error occurred (AccessDenied) when calling the CreatePolicy operation: User: arn:aws:sts::229020249659:assumed-role/AmazonSageMaker-ExecutionRole-20250928T234110/SageMaker is not authorized to perform: iam:CreatePolicy on resource: policy HealthOmicsFullAccess because no identity-based policy allows the iam:CreatePolicy action
